In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cupy as cp

Unified scene definition

In [ ]:
# Common scene parameters
nx, ny = 300, 150
aspect_ratio = nx / ny
fov = np.pi / 3.0
lower_left = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])
origin = np.array([0.0, 0.0, 0.0])
sphere_center = np.array([0.0, 0.0, -1.0])
sphere_radius = 0.5

def ray_color(ro, rd):
    # ray-sphere intersection
    oc = ro - sphere_center
    a = np.dot(rd, rd)
    b = np.dot(oc, rd)
    c = np.dot(oc, oc) - sphere_radius**2
    disc = b*b - a*c
    if disc > 0:
        t = (-b - np.sqrt(disc)) / a
        if t > 0.001:
            hit = ro + t*rd
            normal = (hit - sphere_center) / np.linalg.norm(hit - sphere_center)
            base = np.array([1.0, 1.0, 0.0])
            intensity = 0.5 * (normal[1] + 1.0)
            return intensity * base
    # sky
    udir = rd / np.linalg.norm(rd)
    t = 0.5 * (udir[1] + 1.0)
    return (1-t)*np.array([1.0,1.0,1.0]) + t*np.array([0.5,0.7,1.0])

CUDA kernel

In [ ]:
kernel_code = r'''
extern "C" __global__
void render_kernel(float *fb, int nx, int ny) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i>=nx || j>=ny) return;
    int idx = j*nx + i;
    float u = float(i)/nx;
    float v = float(j)/ny;
    float orig[3] = {0,0,0};
    float ll[3] = {-2,-1,-1};
    float h[3] = {4,0,0};
    float vvec[3] = {0,2,0};
    float sc[3] = {0,0,-1};
    float rad = 0.5f;
    float dir[3] = { ll[0]+u*h[0]+v*vvec[0]-orig[0],
                     ll[1]+u*h[1]+v*vvec[1]-orig[1],
                     ll[2]+u*h[2]+v*vvec[2]-orig[2] };
    float norm = sqrtf(dir[0]*dir[0]+dir[1]*dir[1]+dir[2]*dir[2]);
    dir[0]/=norm; dir[1]/=norm; dir[2]/=norm;
    // intersect
    float oc[3] = {orig[0]-sc[0], orig[1]-sc[1], orig[2]-sc[2]};
    float a = dir[0]*dir[0]+dir[1]*dir[1]+dir[2]*dir[2];
    float b = oc[0]*dir[0]+oc[1]*dir[1]+oc[2]*dir[2];
    float c = oc[0]*oc[0]+oc[1]*oc[1]+oc[2]*oc[2] - rad*rad;
    float disc = b*b - a*c;
    float col[3];
    if (disc>0) {
        float t = (-b - sqrtf(disc))/a;
        if (t>0.001f) {
            float hit[3] = {orig[0]+t*dir[0], orig[1]+t*dir[1], orig[2]+t*dir[2]};
            float nx_ = (hit[0]-sc[0])/rad;
            float ny_ = (hit[1]-sc[1])/rad;
            float nz_ = (hit[2]-sc[2])/rad;
            float intensity = 0.5f*(ny_+1.0f);
            col[0]=intensity; col[1]=intensity; col[2]=0;
        } else disc = -1;
    }
    if (disc<=0) {
        // sky
        float tsky = 0.5f*(dir[1]+1.0f);
        col[0]=(1-tsky)+tsky*0.5f;
        col[1]=(1-tsky)+tsky*0.7f;
        col[2]=(1-tsky)+tsky*1.0f;
    }
    fb[3*idx+0] = col[0];
    fb[3*idx+1] = col[1];
    fb[3*idx+2] = col[2];
}
'''
render_kernel = cp.RawKernel(kernel_code, 'render_kernel')

def render_cupy_kernel():
    fb = cp.zeros(3*nx*ny, dtype=cp.float32)
    threads = (16, 16)
    blocks = ((nx+15)//16, (ny+15)//16)
    # prepare params
    params = (fb, nx, ny,
              *lower_left, *horizontal, *vertical,
              *origin, *sphere_center, sphere_radius)
    start = cp.cuda.Event()
    stop = cp.cuda.Event()
    start.record()
    render_kernel(blocks, threads, params)
    stop.record()
    stop.synchronize()
    elapsed = cp.cuda.get_elapsed_time(start, stop)
    img = fb.reshape(ny, nx, 3).get()
    return img, elapsed

In [ ]:
img, exec_time = render_cupy_kernel()
print(exec_time, "ms")
plt.imshow(img)
plt.axis('off')
plt.show()